In [1]:
import os
import numpy as np
# import torch

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%matplotlib notebook
import matplotlib.pyplot as plt

from point2box.src.myutils import get_scenes_from_dir, make_dataset_from_scenes, MOTDataset_BBwise, row_wise_iou

from point2box.src.myutils import MOTDataset_Imagewise

import cv2
import torchvision

import joblib

import ipywidgets as wdg  # Using the ipython notebook widgets


%load_ext autoreload
%autoreload 2

In [2]:
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader

import coco_eval
import csv_eval


## Notebook Configurations

In [3]:
notebook_config = {}
notebook_config['dataset']='mot'                #'Dataset type, must be one of csv or coco.'
notebook_config['coco_path']='../coco/'                #'Path to COCO directory'
notebook_config['mot_path']='../Datasets/'
notebook_config['point_dist_path']='../point2box/point_dist'
notebook_config['csv_train']=None                #'Path to file containing training annotations (see readme)'
notebook_config['csv_classes']=None                #'Path to file containing class list (see readme)'
notebook_config['csv_val']=None                #'Path to file containing validation annotations (optional, see readme)'
notebook_config['depth']=18                #'Resnet depth, must be one of 18, 34, 50, 101, 152'
notebook_config['epochs']=20                #'Number of epochs'
notebook_config['use_gpu'] = True
notebook_config['batch_size'] = 1
notebook_config['load_sampler'] = True
notebook_config['save_sampler'] = not notebook_config['load_sampler']




## Create DataLoader


In [4]:
# all_scenes = get_scenes_from_dir(notebook_config['mot_path'])
# split_point = int(0.9*len(all_scenes))
# print(len(all_scenes), split_point)
# train_scenes = all_scenes[:split_point]
# val_scenes = all_scenes[split_point:]


In [5]:
# val_scenes

In [6]:
# dataset_train = MOTDataset_Imagewise(train_scenes, path_for_points= notebook_config['point_dist_path'], 
#                                        transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))

# dataset_val = MOTDataset_Imagewise(val_scenes, path_for_points= notebook_config['point_dist_path'], 
#                                      transform=transforms.Compose([Normalizer(), Resizer()]))

In [7]:
# if notebook_config['load_sampler']:
#     sampler =  joblib.load('sampler.pkl')
#     dataloader_train = joblib.load('data_loader.pkl')
#     if dataset_val is not None:
#         sampler_val =  joblib.load('sampler_val.pkl')
#         dataloader_val = joblib.load('dataloader_val.pkl')
# else:
#     sampler = AspectRatioBasedSampler(dataset_train, batch_size=notebook_config['batch_size'], drop_last=False)
#     dataloader_train = DataLoader(dataset_train, num_workers=8, collate_fn=collater, batch_sampler=sampler)
#     if dataset_val is not None:
#         sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
#         dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)

    

In [8]:
# if notebook_config['save_sampler']:
#     joblib.dump(sampler, 'sampler.pkl')
#     joblib.dump(dataloader_train, 'data_loader.pkl')
#     if dataset_val is not None:
#         joblib.dump(sampler_val, 'sampler_val.pkl')
#         joblib.dump(dataloader_val, 'dataloader_val.pkl')


In [9]:
# if notebook_config['depth'] == 18:
#     retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
# elif notebook_config['depth'] == 34:
#     retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
# elif notebook_config['depth'] == 50:
#     retinanet = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=True)
# elif notebook_config['depth'] == 101:
#     retinanet = model.resnet101(num_classes=dataset_train.num_classes(), pretrained=True)
# elif notebook_config['depth'] == 152:
#     retinanet = model.resnet152(num_classes=dataset_train.num_classes(), pretrained=True)
# else:
#     raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')        



# if notebook_config['use_gpu']:
#     retinanet = retinanet.cuda()
#     retinanet = torch.nn.DataParallel(retinanet).cuda()

# retinanet.training = True


In [10]:
# optimizer = optim.Adam(retinanet.parameters(), lr=1e-5)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

# loss_hist = collections.deque(maxlen=500)

# retinanet.train()
# retinanet.module.freeze_bn()
# print('Num training images: {}'.format(len(dataset_train)))

In [11]:
# for i, data in enumerate(dataloader_train):
#     print(i)
#     print(data['img'].shape)
#     if i>5:
#         break

In [12]:
# retinanet.train()
# retinanet.module.freeze_bn()

# epoch_loss = []

In [13]:
# optimizer.zero_grad()

In [14]:
# classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])

In [15]:
# for epoch_num in range(notebook_config['epochs']):

#     retinanet.train()
#     retinanet.module.freeze_bn()

#     epoch_loss = []

#     for iter_num, data in enumerate(dataloader_train):
#         try:
#             optimizer.zero_grad()

#             classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])

#             classification_loss = classification_loss.mean()
#             regression_loss = regression_loss.mean()

#             loss = classification_loss + regression_loss

#             if bool(loss == 0):
#                 continue

#             loss.backward()

#             torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)

#             optimizer.step()

#             loss_hist.append(float(loss))

#             epoch_loss.append(float(loss))

#             print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))

#             del classification_loss
#             del regression_loss
#         except Exception as e:
#             print(e)
#             continue

#     if notebook_config['dataset'] == 'coco':

#         print('Evaluating dataset')

#         coco_eval.evaluate_coco(dataset_val, retinanet)

#     elif notebook_config['dataset'] == 'csv' and notebook_config['csv_val'] is not None:

#         print('Evaluating dataset')

#         mAP = csv_eval.evaluate(dataset_val, retinanet)


#     scheduler.step(np.mean(epoch_loss))    

#     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(notebook_config['dataset'], epoch_num))

# retinanet.eval()

# torch.save(retinanet, 'model_final.pt'.format(epoch_num))


In [16]:
# retinanet = torch.load('mot_retinanet_0.pt')

# retinanet.training = False

In [17]:
# retinanet.cuda()

In [18]:
# dataset_val.length = 100

In [19]:
# retinanet.eval()
# mAP = csv_eval.evaluate(dataset_val, retinanet)

## Rough work

In [20]:
# joblib.dump(dataset_val, 'dataset_val.pkl')
dataset_val = joblib.load('dataset_val.pkl')

In [21]:
dataset_val.get_point = True

In [22]:
sample = dataset_val[0]

(1920, 1080)


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
points = sample['points'].astype(np.int32)
print(points)

In [ ]:
print(sample['img'].shape[:2])
point_channel = np.zeros(sample['img'].shape[:2])
point_channel[points[:, -1::]] = 255
# plt.imshow(sample['img'].numpy()/np.max(sample['img'].numpy()))